In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from funkwpap import *
import sympy as sp, pandas as pd, numpy as np, tqdm, time, sys, matplotlib.pyplot as plt
from scipy.optimize import fsolve
from random import random
from scipy.optimize import minimize
from gurobipy import *

In [2]:
x = sp.symbols('x')
cap = 200 # total emission cap
Regulator91 = Regulator("test1", permit_price = 2.4079, emission_cap = cap)
sector1 = Sector('cement', price_demand_function= 100 - 0.1*x, free_emission_multiplier= 0, regulator= Regulator91)
sector2 = Sector('steel', price_demand_function=150 - 0.1*x, free_emission_multiplier= 0, regulator= Regulator91)
sector3 = Sector('paper', price_demand_function= 200 - 0.2*x, free_emission_multiplier= 0, regulator= Regulator91)
sector4 = Sector('plastic', price_demand_function= 400 - 0.5*x, free_emission_multiplier= 0, regulator= Regulator91)
sector5 = Sector('glass', price_demand_function= 300 - 0.4*x, free_emission_multiplier= 0, regulator= Regulator91)
country1 = Country('DE', 1, regulator= Regulator91)
country2 = Country('FI', 0.5, regulator= Regulator91)
country3 = Country('GR', size= 0.1, regulator= Regulator91)

# Create Firms using objects
firm1 = Firm('firm1', 1, 1, x*0, 10*x+ 2*x**2 + 0.1*x**3 , 0, 0, 0, regulator= Regulator91)
firm2 = Firm('firm2', 1, 2, x*0, 11*x+ 3*x**2 + 0.2*x**3, 0, 0, 0, regulator= Regulator91)
firm3 = Firm('firm3', 1, 3, x*0, 5*x+ 4*x**2 + 5*x**3 , 0, 0, 0, regulator= Regulator91)
firm4 = Firm('firm4', 2, 1, x*0, 7*x+ 5*x**2 + 3*x**3 , 0, 0, 0, regulator= Regulator91)
firm5 = Firm('firm5', 2, 2, x*0, 1*x+ 6*x**2 + 2*x**3 , 0, 0, 0, regulator= Regulator91)
firm6 = Firm('firm6', 2, 3, x*0, 2*x+ 7*x**2 + 3*x**3 , 0, 0, 0, regulator= Regulator91)
firm7 = Firm('firm7', 3, 1, x*0, 3*x+ 8*x**2 + 4*x**3 , 0, 0, 0, regulator= Regulator91)
firm8 = Firm('firm8', 3, 2, x*0, 4*x+ 9*x**2 + 10*x**3 , 0, 0, 0, regulator= Regulator91)
firm9 = Firm('firm9', 3, 3, x*0, 5*x+ 10*x**2 + 11*x**3 , 0, 0, 0, regulator= Regulator91)


In [3]:

# Define one pair of output and emission for each firm for sympy and for gurobi and the dictionary of them
sympy_output = {}
sympy_emission = {}
sympy_abatement = {}

# ab = abatement ab = q - x (output - emission)

for firm in Regulator91.firm_registry.values():
    q_sym = sp.symbols(f"q{firm.id}")
    x_sym = sp.symbols(f"x{firm.id}")
    ab_sym = sp.symbols(f"ab{firm.id}")
    sympy_output[firm.id] = q_sym
    sympy_emission[firm.id] = x_sym
    sympy_abatement[firm.id] = ab_sym
    

pp = sp.symbols('pp')  # Permit price

        
# Define the objective function
sympy_objective = 0
for firm in Regulator91.firm_registry.values():
    firm_profit = 0
    sect = firm.sector
    sum_sector_outputs = 0
    for i in range(len(sect.firms)):
        sum_sector_outputs += sympy_output[sect.firms[i].id]
    firm_revenew = sect.price_demand_function.subs(x, sum_sector_outputs) * sympy_output[firm.id]
    firm_abatement = -firm.abatement_cost_function.subs(x, sympy_abatement[firm.id])
    firm_trading = -pp * ((1 - sect.free_emission_multiplier) * sympy_output[firm.id] - sympy_abatement[firm.id])
    firm_profit += firm_revenew + firm_abatement + firm_trading
    profit_dq = sp.diff(firm_profit, sympy_output[firm.id])
    profit_dab = sp.diff(firm_profit, sympy_abatement[firm.id])
    # sympy_objective += profit_dq + profit_dab
    sympy_objective += - profit_dq - profit_dab

print(sympy_objective)
#print the Hessian matrix and the eigenvalues
variables = list(sympy_output.values()) + list(sympy_abatement.values())
Hessian = sp.hessian(sympy_objective, variables)
print(Hessian)
eigenvalues = Hessian.eigenvals()
print(eigenvalues)

# If all eigenvalues are non negative, the Hessian is positive semi-definite and the objective function is convex

0.3*ab1**2 + 4*ab1 + 0.6*ab2**2 + 6*ab2 + 15*ab3**2 + 8*ab3 + 9*ab4**2 + 10*ab4 + 6*ab5**2 + 12*ab5 + 9*ab6**2 + 14*ab6 + 12*ab7**2 + 16*ab7 + 30*ab8**2 + 18*ab8 + 33*ab9**2 + 20*ab9 + 0.4*q1 + 0.4*q2 + 0.4*q3 + 0.4*q4 + 0.4*q5 + 0.4*q6 + 0.8*q7 + 0.8*q8 + 0.8*q9 - 1302
Matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0.600000000000000, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.20000000000000, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 30, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0

In [8]:
m = Regulator91.optimization_concave_formulation_ab(gurobi_print= False, print_output = False)
m.display()
'''Minimize
-1302.0 + 0.4 qq1 + 4.0 ab1 + 0.4 qq2 + 6.0 ab2 + 0.4 qq3 + 8.0 ab3 + 0.4 qq4
+ 10.0 ab4 + 0.4 qq5 + 12.0 ab5 + 0.4 qq6 + 14.0 ab6 + 0.8 qq7 + 16.0 ab7 + 0.8 qq8
+ 18.0 ab8 + 0.8 qq9 + 20.0 ab9 + [ 0.30000000000000004 ab1 ^ 2
+ 0.6000000000000001 ab2 ^ 2 + 15.0 ab3 ^ 2 + 9.0 ab4 ^ 2 + 6.0 ab5 ^ 2 + 9.0 ab6 ^ 2
+ 12.0 ab7 ^ 2 + 30.0 ab8 ^ 2 + 33.0 ab9 ^ 2 ]
Subject To
  R0: qq1 + -1.0 ab1 >= 0
  R1: ab1 >= 0
  R2: qq2 + -1.0 ab2 >= 0
  R3: ab2 >= 0
  R4: qq3 + -1.0 ab3 >= 0
  R5: ab3 >= 0
  R6: qq4 + -1.0 ab4 >= 0
  R7: ab4 >= 0
  R8: qq5 + -1.0 ab5 >= 0
  R9: ab5 >= 0
  R10: qq6 + -1.0 ab6 >= 0
  R11: ab6 >= 0
  R12: qq7 + -1.0 ab7 >= 0
  R13: ab7 >= 0
  R14: qq8 + -1.0 ab8 >= 0
  R15: ab8 >= 0
  R16: qq9 + -1.0 ab9 >= 0
  R17: ab9 >= 0
  R18: -0.2 qq1 + -0.1 qq2 + -0.1 qq3 + -1.0 ppp <= -100
  R19: -0.1 qq1 + -0.2 qq2 + -0.1 qq3 + -1.0 ppp <= -100
  R20: -0.1 qq1 + -0.1 qq2 + -0.2 qq3 + -1.0 ppp <= -100
  R21: -0.2 qq4 + -0.1 qq5 + -0.1 qq6 + -1.0 ppp <= -150
  R22: -0.1 qq4 + -0.2 qq5 + -0.1 qq6 + -1.0 ppp <= -150
  R23: -0.1 qq4 + -0.1 qq5 + -0.2 qq6 + -1.0 ppp <= -150
  R24: -0.4 qq7 + -0.2 qq8 + -0.2 qq9 + -1.0 ppp <= -200
  R25: -0.2 qq7 + -0.4 qq8 + -0.2 qq9 + -1.0 ppp <= -200
  R26: -0.2 qq7 + -0.2 qq8 + -0.4 qq9 + -1.0 ppp <= -200
R27: qq1 + -1.0 ab1 + qq2 + -1.0 ab2 + qq3 + -1.0 ab3 + qq4 + -1.0 ab4 + qq5 + -1.0 ab5
 + qq6 + -1.0 ab6 + qq7 + -1.0 ab7 + qq8 + -1.0 ab8 + qq9 + -1.0 ab9 = 1000
  : -4.0 ab1 + ppp + [ -0.30000000000000004 ab1 ^ 2 ] <= 10
  : -6.0 ab2 + ppp + [ -0.6000000000000001 ab2 ^ 2 ] <= 11
  : -8.0 ab3 + ppp + [ -15.0 ab3 ^ 2 ] <= 5
  : -10.0 ab4 + ppp + [ -9.0 ab4 ^ 2 ] <= 7
  : -12.0 ab5 + ppp + [ -6.0 ab5 ^ 2 ] <= 1
  : -14.0 ab6 + ppp + [ -9.0 ab6 ^ 2 ] <= 2
  : -16.0 ab7 + ppp + [ -12.0 ab7 ^ 2 ] <= 3
  : -18.0 ab8 + ppp + [ -30.0 ab8 ^ 2 ] <= 4
  : -20.0 ab9 + ppp + [ -33.0 ab9 ^ 2 ] <= 5
  '''

Optimal solution found


C:\Users\Kostas\AppData\Local\Temp\ipykernel_375952\1013396723.py:2: DeprecationWarning: Model.display() is deprecated
  m.display()


**CHATGPT ANALYSIS**

### **Objective Function:**
You are minimizing a quadratic function, which is a sum of linear and quadratic terms. The quadratic terms appear to be positive, such as:

\[
\text{Minimize: } -1302 + 0.4q_1 + 4.0ab_1 + 0.4q_2 + 6.0ab_2 + \ldots + [0.3ab_1^2 + 0.6ab_2^2 + 15ab_3^2 + \ldots]
\]
- Quadratic terms are positive coefficients, which indicates the objective function is **convex**. In convex minimization, the objective should have non-negative second derivatives (positive or zero quadratic terms).

### **Constraints:**
You have a mix of linear inequalities and quadratic constraints. Let's break them down:

1. **Linear Inequalities**:
   - Constraints like:
     \[
     qq_1 - ab_1 \geq 0, \quad ab_1 \geq 0, \quad \text{and so on for each } ab_i
     \]
     These are simple linear constraints, which are convex.

2. **Quadratic Constraints**:
   You have several quadratic constraints like:
   \[
   -4.0ab_1 + ppp + [-0.3ab_1^2] \leq 10
   \]
   Here, the quadratic terms like \(-0.3ab_1^2\) are **negative**, meaning they contribute non-convex terms. Negative quadratic terms are concave, and their presence in inequality constraints makes these constraints non-convex.

   Specifically, for a convex minimization, the quadratic terms in the inequality constraints should be convex (i.e., positive or zero). Since your constraints have **negative quadratic terms**, they introduce **non-convexity** into the problem.

### **Conclusion:**
- Your **objective function** is convex and suitable for minimization.
- However, your **quadratic constraints** introduce **non-convexity** due to the negative quadratic terms like \(-0.3ab_1^2\). These constraints are likely causing Gurobi to flag the problem as non-convex.
